# Libraries import

Importing basic libraries for the code execution.

In [1]:
import time

# Sudoku class

A fundamental class, which is in principle an array with an aditional feature to work quickly with the inner squares.

In [2]:
class Sudoku():
    # First index is row, second is collumn, starting from zero
    def __init__(self, array):
        self.table = array
        # create a list of collumns
        self.collumns = []
        for a in range(9):
            self.collumns.append([val[a] for val in self.table])
        # create a list of inner squares
        self.squares = []
        # every element in the square list consists of elements in a suqare, counted in the reading order, i.e. square[0] is tope left square, square[1] is top middle square etc.
        for a in range(9):
            # create an empty square at first
            temp_square = []
            # add all 3x3 elements of the square, based on its position a.
            for b in range(3):
                for c in range(3):
                    # only unique, non-zero values are recorded in the square to avoid repetition
                    if self.table[(a // 3)*3 + b][(a % 3)*3 + c] != 0 and self.table[(a // 3)*3 + b][(a % 3)*3 + c] not in temp_square:
                        temp_square.append(self.table[(a // 3)*3 + b][(a % 3)*3 + c])
            #print('Square created:', temp_square)
            self.squares.append(temp_square)
    
    def __repr__(self):
        # Creating a string representation of the whole sudoku
        print_string = ''
        # Two FOR loops which go through all elements of the sudoku table and making them into a string, with lines to make it easier to visualise the table
        for a in range(9):
            for b in range(9):
                print_string = print_string+str(self.table[a][b])
                # Adding a vertical line to create inside squares
                if b % 3 == 2:
                    print_string = print_string+'|'
            print_string = print_string+'\n'
            # adding vertical line to create inside squares
            if a % 3 == 2:
                print_string = print_string+'------------\n'
        # Returning the final string
        return str(print_string)
    
    # Method for allocating a new number in a sudoku
    def allocate_number(self, row, collumn, number):
        # Add a number to the sudoku table, which will take care of the row as well.
        self.table[row][collumn] = number
        # Add the number to the appropriate collumn
        self.collumns[collumn][row] = number
        # Add the number to the appropriate square
        self.squares[(row // 3)*3 + collumn // 3].append(number)
    
    # Method for removing a number allocated from sudoku.
    def remove_number(self, row, collumn, number):
        # Remove the number from the sudoku table, i.e. set it to zero
        self.table[row][collumn] = 0
        # Remove the number from the appropriate collumn
        self.collumns[collumn][row] = 0
        # Remove the number from the appropriate square
        self.squares[(row // 3)*3 + collumn // 3].remove(number)
        
    

# Leaf class

It is a class which will be holding possible allocations to each cell of sudoku, with additional methods to allow for the execution of branch & bound algorythm.

In [3]:
class Leaf():
    # Defining the leaf, which has a position in sudoku, defined by the row and collumn, and the possible allocations
    def __init__(self, row, collumn, possible_allocations):
        self.position = [row, collumn]
        self.possible_allocations = possible_allocations
    
    def __repr__(self):
        return 'Position: '+str(self.position)+'\n'+'Possible allocations: '+str(self.possible_allocations)+'\n'
    
    # a recursive method which will execute the branch and bound algorythm
    def branch_and_bound(self, sudoku, leaf_list, leaf_list_position, finished):
        # Finish the algorythm if the leaf_list_position is larger than the leaf_list length- the sudoku is finished
        #if leaf_list_position == len(leaf_list):
            #return
        #print('current leaf:', self)
        #print('position in the leaf_list:', leaf_list_position)
        # Go through each possible number allocation to a given cell
        for number in self.possible_allocations:
            #print('Currently tested number:', number)
            # If the tested number is acceptable...
            if acceptance_test(sudoku, self.position[0], self.position[1], number) == True:
                # ... Make this allocation to the sudoku
                sudoku.allocate_number(self.position[0], self.position[1], number)
                #print('number', number, 'accepted')
                #print(sudoku)
                # Finish the algorythm if the leaf_list_position is larger than the leaf_list length- the sudoku is finished
                if leaf_list_position+1 == len(leaf_list):
                    #print('return activated')
                    return True               
                # Call the branch and bound algorythm of the next leaf
                finished = leaf_list[leaf_list_position+1].branch_and_bound(sudoku, leaf_list, leaf_list_position+1, finished)
                if finished == True:
                    #print('return activated')
                    return True
                # If this code is executed, this means that there were no acceptable allocations in the current leaf, so the current allocation here needs to be removed
                sudoku.remove_number(self.position[0], self.position[1], number)
                #print(sudoku)
            #print('number', number, 'rejected')
        #print('no acceptable allocations, retuning to the previous leaf')
        return False
        # If there are no legal allocations, that means that an error was made previously, decrease the leaf_list_position and finish this instance of branch and bound, which will go back to the previous leaf
                

# Acceptance test function

It tests if a particular position in a particular sudoku can take a particular value.

In [4]:
def acceptance_test(used_sudoku, row, collumn, tested_number):
    # Variable decribing if the tested number can be accepted for the tested position.
    accept = True
    # Check if the tested number is in the row and change the acceptance accordingly.
    if tested_number in used_sudoku.table[row][:]:
        accept = False # print('Tested number already in the row')
    # Check if the tested number is in the collumn
    if tested_number in used_sudoku.collumns[collumn]:
        accept = False # print('Tested number already in the collumn')
    # Check if the tested number is in the square
    if tested_number in used_sudoku.squares[(row // 3) * 3 + collumn // 3]:
        accept = False # print('Tested number already in the square')
    return accept

# Possible number allocation function

It just takes the acceptance_test function, runs it 9 times over all possible digits and returns the possible numbers allocations.

In [5]:
def possible_numbers_allocation(used_sudoku, row, collumn):
    # creating a list of possible numbers in given position
    possible_numbers = []
    # run the acceptance test 9 times, and when it returns TRUE, record the number
    for a in range(1,10):
        #print(a)
        temp_acceptance = acceptance_test(used_sudoku, row, collumn, a)
        #print(temp_acceptance)
        if temp_acceptance == True:
            possible_numbers.append(a)
            #print('current list:', possible_numbers)
    # return the list of all possible numbers
    return possible_numbers

# Initial leafs creation function

This function uses possible_number_allocation_function to run over all positions, and when they have zeros on them (i.e. have no allocation yet), it test what can go there. It returns a list of leafs, ordered according to the number of possible allocations.

In [6]:
def intial_leafs_creation(used_sudoku):
    # create an empty list, for all leafs
    leaf_list = []
    # go through all cells in the sudoku, when they are marked as zero, run possible_number allocation to create a leaf for this position in sudoku
    for a in range(9):
        for b in range(9):
            if used_sudoku.table[a][b] == 0:
                leaf_list.append(Leaf(a,b,possible_numbers_allocation(used_sudoku,a,b)))
    # SORT THE LIST
    leaf_list.sort(key=lambda x: len(x.possible_allocations))
    # Return the sorted list of leafs
    return leaf_list
    

# Sudoku input and solution

In [16]:
test_sudoku = Sudoku([[2,0,0,0,8,5,0,0,4],
                      [0,0,0,7,0,3,0,9,0],
                      [0,0,0,4,0,0,0,0,8],
                      [0,0,0,0,0,0,2,4,0],
                      [5,0,0,0,0,0,0,0,1],
                      [0,2,7,0,0,0,0,0,0],
                      [4,6,0,0,0,2,0,0,0],
                      [0,1,0,6,7,9,0,0,0],
                      [3,0,0,5,4,0,0,0,9]])
print('Starting Sudoku:')
print(test_sudoku)
test_leaf_list = intial_leafs_creation(test_sudoku)
#print('Length of the leaf list:', len(test_leaf_list))
# Variable for defining on which leaf on the leaf list we are
leaf_list_position = 0
start = time.time()
# Starting the recursive branch and bound leaf method
solved = test_leaf_list[leaf_list_position].branch_and_bound(test_sudoku, test_leaf_list, leaf_list_position, False)
end = time.time()
# show the results if the sudoku has a solution
if solved == True:    
    print('Solved Sudoku:')
    print(test_sudoku)
else:
    print('Sudoku has no solution')
# printing the final run time
run_time = end - start
print('Running time:', run_time)

Starting Sudoku:
200|085|004|
000|703|090|
000|400|008|
------------
000|000|240|
500|000|001|
027|000|000|
------------
460|002|000|
010|679|000|
300|540|009|
------------

Solved Sudoku:
293|185|764|
648|723|195|
751|496|328|
------------
186|957|243|
534|268|971|
927|314|856|
------------
469|832|517|
815|679|432|
372|541|689|
------------

Running time: 0.038884878158569336
